In [54]:
from sympy import *

In [55]:
#definição de variáveis
# DV = (delta)V
Vcc, s, L, Iof, I, R, Vdo, Ioa, Vra, Vfr, t, k, T, V0, V1, DV = symbols('Vcc s L Iof I R Vdo Ioa Vra Vfr t k T V0 V1 DV')

In [56]:
# chave aberta
Eq1=Eq(Vcc/s -s*L*I + L*Iof - R*I,0)
Eq1

Eq(-I*L*s - I*R + Iof*L + Vcc/s, 0)

In [57]:
# chave fechada
Eq2=Eq(-Vdo/s-s*L*I+Ioa - R*I,0)
Eq2

Eq(-I*L*s - I*R + Ioa - Vdo/s, 0)

In [58]:
# define a corrente no circuito com chave fechada
sol1 = solve(Eq1,I)
sol1

[(Iof*L*s + Vcc)/(s*(L*s + R))]

In [59]:
# define a tensão no resistor no circuito com chave fechada
Vrf=sol1[0]*R
Vrf

R*(Iof*L*s + Vcc)/(s*(L*s + R))

In [60]:
# define a corrente no circuito com chave aberta
sol2 = solve(Eq2,I)
sol2

[(Ioa*s - Vdo)/(s*(L*s + R))]

In [61]:
# define a tensão no resistor no circuito com chave aberta
Vra=sol2[0]*R
Vra

R*(Ioa*s - Vdo)/(s*(L*s + R))

In [62]:
# Simplificar as expressões e depois expandir em frações parciais
VRf_s_simplified = simplify(Vrf)
VRa_s_simplified = simplify(Vra)

# Tentativa de expansão em frações parciais
VRf_s_fractioned = apart(VRf_s_simplified, s)
VRa_s_fractioned = apart(VRa_s_simplified, s)

VRf_s_fractioned, VRa_s_fractioned

(L*(Iof*R - Vcc)/(L*s + R) + Vcc/s, -Vdo/s + (Ioa*R + L*Vdo)/(L*s + R))

In [63]:
VRf_s_fractioned.as_ordered_terms(),VRa_s_fractioned.as_ordered_terms()

([L*(Iof*R - Vcc)/(L*s + R), Vcc/s], [-Vdo/s, (Ioa*R + L*Vdo)/(L*s + R)])

In [64]:
# Calcular os residuos
# Polos e residuos para Vrf(s)
polos_residuos_VRf = [(p, residue(VRf_s_fractioned, s, p)) for p in VRf_s_fractioned.as_ordered_terms()]

# Polos e resíduos para VRa(s)
polos_residuos_VRa = [(p, residue(VRa_s_fractioned, s, p)) for p in VRa_s_fractioned.as_ordered_terms()]

polos_residuos_VRf, polos_residuos_VRa

([(L*(Iof*R - Vcc)/(L*s + R), 0), (Vcc/s, 0)],
 [(-Vdo/s, 0), ((Ioa*R + L*Vdo)/(L*s + R), 0)])

In [65]:
# Calculando a transformada inversa de Laplace apenas para VRf(s)
vRf_t = inverse_laplace_transform(VRf_s_fractioned, s, t)
vRa_t = inverse_laplace_transform(VRa_s_fractioned, s, t)

vRf_t,vRa_t

(Vcc*Heaviside(t) + (Iof*R - Vcc)*exp(-R*t/L)*Heaviside(t),
 -Vdo*Heaviside(t) + (Ioa*R + L*Vdo)*exp(-R*t/L)*Heaviside(t)/L)

## Análise no Tempo

In [66]:
# Aplicaremos a transformada inversa de Laplace para obter a tensão no domínio do tempo
# Dividimos por Heaviside(t) para elimitar o Euler como função de teta

Vrft = simplify(inverse_laplace_transform(Vrf, s, t))/Heaviside(t)
Vrft

(Iof*R + Vcc*exp(R*t/L) - Vcc)*exp(-R*t/L)

In [67]:
# Aplicaremos a transformada inversa de Laplace para obter a tensão no domínio do tempo
# Dividimos por Heaviside(t) para elimitar o Euler como função de teta

Vrat = simplify(inverse_laplace_transform(Vra, s, t))/Heaviside(t)
Vrat

(Ioa*R - L*Vdo*exp(R*t/L) + L*Vdo)*exp(-R*t/L)/L

## V1 e V2

In [68]:
# substituindo os valores de em t e na corrente inicial (condição inicial) para definir V1
V1=Vrft.subs(t, k*T)
V1=V1.subs(Iof, V0/R)
V1

(V0 + Vcc*exp(R*T*k/L) - Vcc)*exp(-R*T*k/L)

In [69]:
# substituindo os valores de em t e na corrente inicial (condição inicial) para definir V2

V2=Vrat.subs(t, (1-k)*T)
V2=V2.subs(Ioa, V1/R)
V2

(-L*Vdo*exp(R*T*(1 - k)/L) + L*Vdo + (V0 + Vcc*exp(R*T*k/L) - Vcc)*exp(-R*T*k/L))*exp(-R*T*(1 - k)/L)/L

## Análise em regime permanente

In [70]:
# Achar os valores de V0 e DV
# Considerando V2 = V0 e V1 = V0 + DV

# V0 + DV = V1
prop1 = Eq(V0 + DV, V1)

# V0 = V2
prop2 = Eq(V0, V2)

# solucionando o sistema e obtendo os valores de V0 e DV
Vsol = linsolve([prop1, prop2], (V0, DV))
Vsol.args[0][1]

L*Vcc*exp(R*T/L)*exp(R*T*k/L)/(L*exp(R*T/L)*exp(R*T*k/L) - exp(R*T*k/L)) - L*Vcc*exp(R*T/L)/(L*exp(R*T/L)*exp(R*T*k/L) - exp(R*T*k/L)) + L*Vdo*exp(R*T/L)*exp(R*T*k/L)/(L*exp(R*T/L)*exp(R*T*k/L) - exp(R*T*k/L)) - L*Vdo*exp(R*T/L)/(L*exp(R*T/L)*exp(R*T*k/L) - exp(R*T*k/L)) - L*Vdo*exp(2*R*T*k/L)/(L*exp(R*T/L)*exp(R*T*k/L) - exp(R*T*k/L)) + L*Vdo*exp(R*T*k/L)/(L*exp(R*T/L)*exp(R*T*k/L) - exp(R*T*k/L)) - Vcc*exp(2*R*T*k/L)/(L*exp(R*T/L)*exp(R*T*k/L) - exp(R*T*k/L)) + Vcc*exp(R*T*k/L)/(L*exp(R*T/L)*exp(R*T*k/L) - exp(R*T*k/L))

## Série de Taylor e limites

In [71]:
# aplicando a série de Taylor e os limites
tayDV = series(Vsol.args[0][1], T, 0, 2)
tayV0 = series(Vsol.args[0][0], T, 0, 2)
tayDV

T*(R*Vcc*k/(L - 1) - R*Vcc*k/(L*(L - 1))) + O(T**2)

In [72]:
# Extraindo os termos da série (normalmente o termo constante e o coeficiente de T)
V0_linear = tayV0.removeO()  # Remove os termos de ordem superior
V_linear = tayDV.removeO()    

# Valores de V0 e V quando T -> 0
V0_at_T0 = V0_linear.subs(T, 0)
V_at_T0 = V_linear.subs(T, 0)

tayV0

T*(R*Vdo*k/(L - 1) - R*Vdo/(L - 1) + R*Vcc*k/(L*(L - 1))) + O(T**2)

In [73]:
# Define a condição de V ser 0.001 * Vcc
condicao = Eq(tayV0, 0.001 * Vcc)
eq_condicao = condicao.subs(tayV0, V_linear)
sol_L = solve(eq_condicao,L)[0]

sol_L

1000.0*R*T*k

In [74]:

#R = 5 ohm
#Feq = 10 kHz
L1=sol_L.subs([(k, 1/2), (R,5), (T, 1/10000)])

#Feq = 50 kHz
L2=sol_L.subs([(k, 1/2), (R,5), (T, 1/50000)])
L1,L2

(0.250000000000000, 0.0500000000000000)

In [75]:
#R = 1 ohm
#Feq = 10 kHz
L3=sol_L.subs([(k, 1/2), (R,1), (T, 1/10000)])

#Feq = 50 kHz
L4=sol_L.subs([(k, 1/2), (R,1), (T, 1/50000)])
L3, L4

(0.0500000000000000, 0.0100000000000000)

In [77]:
import numpy as np
import sympy as sp

# Definindo as variáveis simbólicas
V, Vcc, L, R, f = sp.symbols('V Vcc L R f')

# Definição dos valores
Rs = [5, 1]  # Valores de R
frequencies = [10e3, 50e3]  # Frequências em Hz
Vcc_value = 5  # Valor de Vcc, por exemplo 5V

# Defina a condição de V ser 0.001 * Vcc
condicao = sp.Eq(V, 0.001 * Vcc)

# Cálculo de L para cada combinação de R e f
results = {}
for R_value in Rs:
    for f_value in frequencies:
        # Substitui Vcc, R e f na equação
        V_linear = -R_value / (2 * sp.pi * f_value * L)  # Por exemplo, uma fórmula de V de acordo com L, R e f
        eq_condicao = condicao.subs({V: V_linear, Vcc: Vcc_value})
        
        # Resolve a equação para L
        sol_L = sp.solve(eq_condicao, L)
        
        # Armazena o resultado
        results[(R_value, f_value)] = sol_L

results


{(5, 10000.0): [-0.0159154943091895],
 (5, 50000.0): [-0.00318309886183791],
 (1, 10000.0): [-0.00318309886183791],
 (1, 50000.0): [-0.000636619772367581]}